In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [817]:
train = pd.read_csv('data/train_9_11.csv')
weather = pd.read_csv('data/weather_3_12.csv')

In [818]:
len(train),len(test)

(60578, 24552)

In [819]:
# 是否工作日，周末
train['isweekend'] = train['dayofweek'].apply(lambda x: 1 if x==5 or x==6 else 0)
train['isweekday'] = 1 - train['isweekend']
train['day'] = train['ymd'].apply(lambda x: int(x[-2:]))
train['month'] = train['ymd'].apply(lambda x: int(x[5:7]))

In [820]:
train = pd.merge(train, weather, on=['ymd'], how='left')

In [821]:
# 地点小时均值，带衰减权重
def getmean(df):
    num = df['num'].values
    weights = [ 0.85**i for i in range(len(num),0,-1) ] 
    return sum(num*weights)/sum(weights)
train = pd.merge(train,\
train.loc[(train['ymd']<'2017-11-24')&(train.month.isin([10,11])),:].groupby(['loc_id', 'hour']).apply(getmean).reset_index().rename(columns={0:'loc_hour_mean'}),\
                on=['loc_id', 'hour'], how='left')

In [822]:
# 地点星期小时均值，带衰减权重
train = pd.merge(train,\
train.loc[(train['ymd']<'2017-11-24')&(train.month.isin([10,11])),:].groupby(['loc_id', 'dayofweek', 'hour']).apply(getmean).reset_index().rename(columns={0:'loc_week_hour_mean'}),\
                on=['loc_id', 'dayofweek', 'hour'], how='left')

In [823]:
# 小时分区
hour_bins = [-1, 6,19, 23]
train['hour_cut'] = pd.cut(train.hour, hour_bins, labels=False)

In [824]:
# 月份分区，上、中、下旬
day_bins = [0, 10, 20, 31]
train['day_cut'] = pd.cut(train.day, day_bins, labels=False)

In [825]:
# 地点简单分类，食堂，教学楼，公寓
train['loc_type'] = 0
train.loc[train['loc_id'].isin([8,10,12,29]),'loc_type'] = 1  # 食堂
train.loc[train['loc_id'].isin([16,33,27,15,22,24,14,20]),'loc_type'] = 2 # 教室

In [827]:
features = [
            'loc_id','loc_type',
            'month','day','day_cut',
            'hour','hour_cut',
            'dayofweek','isweekday', 'isweekend',
            'loc_hour_mean','loc_week_hour_mean',
            'maxTem','minTem',
            'wea','pm','rain'
             ]
label = ['num']

In [895]:
# 仅用11月数据训练模型
X_train = train.loc[(train['ymd']<'2017-11-24')&(train.month.isin([11])), features]
X_valid = train.loc[(train['ymd']>'2017-11-23'), features]
y_train = train.loc[(train['ymd']<'2017-11-24')&(train.month.isin([11])), label].num.values
y_valid = train.loc[(train['ymd']>'2017-11-23'), label].num.values
print(X_train.shape)
print(X_valid.shape)

(17088, 17)
(5416, 17)


In [896]:
lgb_params = {
        'boosting': 'gbrt',
        'max_depth': 10,
        'num_leaves': 60,
        'learning_rate': 0.01,
        'colsample_bytree': 0.5,
        'objective': 'regression',
        'metric': 'rmse',
        'min_data_in_leaf':10,
        'verbose': -1,
        'nthread': 12,
    }

d_train = lgb.Dataset(X_train, label=y_train)
d_valid = lgb.Dataset(X_valid, label=y_valid)
watchlist = [d_train, d_valid]

model = lgb.train(lgb_params, train_set=d_train, num_boost_round=2000, valid_sets=watchlist, \
early_stopping_rounds=20, verbose_eval=100)
lgb_pred = model.predict(X_valid)

Training until validation scores don't improve for 20 rounds.
[100]	training's rmse: 377.424	valid_1's rmse: 335.874
[200]	training's rmse: 188.941	valid_1's rmse: 173.94
[300]	training's rmse: 134.392	valid_1's rmse: 157.952
Early stopping, best iteration is:
[281]	training's rmse: 140.182	valid_1's rmse: 157.398


## blend

In [42]:
def rmse(y, y_pred):
    assert len(y) == len(y_pred)
    return (sum((y_pred - y)** 2.0)/len(y)) ** 0.5

In [897]:
%%time
rf = RandomForestRegressor(n_estimators=200, max_depth=8, max_features='sqrt', n_jobs=16)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_valid)
print('rf rmse:', rmse(y_valid, rf_pred))

rf rmse: 162.042359538
CPU times: user 3.82 s, sys: 96 ms, total: 3.92 s
Wall time: 779 ms


In [898]:
%%time
gbm = GradientBoostingRegressor(learning_rate=0.01,
                                n_estimators=1000,
                                max_depth=6,
                                max_features='sqrt',
                                criterion='mse',
                               )
gbm.fit(X_train, y_train)
gbm_pred = gbm.predict(X_valid)
print('gbm rmse:', rmse(y_valid, gbm_pred))

gbm rmse: 166.579781058
CPU times: user 7.81 s, sys: 4 ms, total: 7.81 s
Wall time: 7.81 s


In [899]:
rule_ans = train.loc[(train['ymd']<'2017-11-24')&(train.month.isin([10,11]))].groupby(['loc_id', 'dayofweek', 'hour']).\
    apply(getmean).reset_index().rename(columns={0:'num_1'})

In [900]:
rule_pred = pd.merge(X_valid, rule_ans, on=['loc_id','dayofweek','hour'], how='left').fillna(0)['num_1'].values

In [901]:
rmse(y_valid, rule_pred)

165.04012540634545

In [907]:
%%time
# 使用验证集的预测结果来确定模型融合的参数
blend_train= []
res_list = []
weights_list = []
blend_train.append(lgb_pred)
blend_train.append(rf_pred)
blend_train.append(gbm_pred)
blend_train.append(rule_pred)
blend_train = np.array(blend_train)

def rmse_min_func(weights):
    final_prediction = 0
    for weight, prediction in zip(weights, blend_train):
        final_prediction += weight * prediction
    return np.sqrt(mean_squared_error(y_valid, final_prediction))

for k in range(10):
    starting_values = np.random.uniform(size=len(blend_train))
    bounds = [(-1, 1)] * len(blend_train)

    res = minimize(rmse_min_func,
                   starting_values,
                   method='L-BFGS-B',
                   bounds=bounds,
                   options={'disp': False,
                            'maxiter': 100000})
    res_list.append(res['fun'])
    weights_list.append(res['x'])
    print('{iter}\tScore: {score}\tWeights: {weights}'.format(
        iter=(k + 1),
        score=res['fun'],
        weights='\t'.join([str(item) for item in res['x']])))

1	Score: 152.63792965208395	Weights: 0.308056626579	0.214292326739	0.466847895059	-0.0259527494745
2	Score: 152.63792964875674	Weights: 0.30803580382	0.214316163276	0.466843889368	-0.0259513303562
3	Score: 152.6379296485751	Weights: 0.308042023479	0.214322172691	0.466837752444	-0.0259570704885
4	Score: 152.63792964854653	Weights: 0.308065722588	0.214311739818	0.466839792867	-0.0259720373433
5	Score: 152.63792964884126	Weights: 0.308058397746	0.214303473549	0.466860322223	-0.0259775902444
6	Score: 152.63792964563828	Weights: 0.308062261321	0.214300870092	0.466846752795	-0.0259647723524
7	Score: 152.6379296459248	Weights: 0.308054773416	0.214302164733	0.466853937917	-0.0259660818549
8	Score: 152.6379297114536	Weights: 0.308093237842	0.214272745909	0.466790533181	-0.0259084699497
9	Score: 152.63792981023124	Weights: 0.308052947979	0.214425326235	0.466740898378	-0.0259728978956
10	Score: 152.63792964543654	Weights: 0.308058214038	0.214300177357	0.466848467979	-0.0259617996582
CPU times: us

## submit

In [908]:
# 重新训练一次模型，使用11月全部数据
d_train = lgb.Dataset(train.loc[train.month.isin([11]), features], label=train.loc[train.month.isin([11])].num.values)
watchlist = [d_train]
model = lgb.train(lgb_params, train_set=d_train, num_boost_round=300, valid_sets=watchlist, \
early_stopping_rounds=None, verbose_eval=100)

rf.fit(train.loc[train.month.isin([11]), features], \
       train.loc[train.month.isin([11])].num.values)
gbm.fit(train.loc[train.month.isin([11]), features], \
        train.loc[train.month.isin([11])].num.values)

[100]	training's rmse: 375.926
[200]	training's rmse: 191.7
[300]	training's rmse: 139.17


In [871]:
test = pd.read_csv('data/test_data_12.csv')

In [872]:
test['hour'] = test.time_stamp.apply(lambda x: int(x[-2:]))
test['month'] = test.time_stamp.apply(lambda x: int(x.split('-')[1]))
test['day'] = test.time_stamp.apply(lambda x: int(x.split('-')[2][:2]))

In [873]:
test['dayofweek'] = pd.DatetimeIndex(test.time_stamp).dayofweek
test['isweekend'] = test['dayofweek'].apply(lambda x: 1 if x==5 or x==6 else 0)
test['isweekday'] = 1 - test['isweekend']
test['ymd'] = test.time_stamp.apply(lambda x: x[:10])
test = pd.merge(test, weather, on='ymd', how='left')

In [874]:
test = pd.merge(test,\
                train.loc[train.month.isin([10,11])].groupby(['loc_id', 'hour']).apply(getmean).reset_index().rename(columns={0:'loc_hour_mean'}),\
                on=['loc_id', 'hour'], how='left')

In [875]:
test = pd.merge(test,\
                train.loc[train.month.isin([10,11])].groupby(['loc_id', 'dayofweek', 'hour']).apply(getmean).reset_index().rename(columns={0:'loc_week_hour_mean'}),\
                on=['loc_id', 'dayofweek', 'hour'], how='left')

In [876]:
hour_bins = [-1, 6, 19, 23]
test['hour_cut'] = pd.cut(test.hour, hour_bins, labels=False)
day_bins = [0, 10, 20, 31]
test['day_cut'] = pd.cut(test.day, day_bins, labels=False)

In [877]:
test['loc_type'] = 0
test.loc[test['loc_id'].isin([8,10,12,29]),'loc_type'] = 1
test.loc[test['loc_id'].isin([16,33,27,15,22,24,14,20]),'loc_type'] = 2

In [878]:
rule_ans_test = train.loc[train.month.isin([10,11])].groupby(['loc_id', 'dayofweek', 'hour']).\
apply(getmean).reset_index().rename(columns={0:'num_1'})

In [879]:
rule_res = \
pd.merge(test, rule_ans_test, on=['loc_id','dayofweek','hour'], how='left')['num_1'].values

In [909]:
%%time
test['num_of_people'] = \
weights_list[-1][0] * model.predict(test[features]) +\
weights_list[-1][1] * rf.predict(test[features]) +\
weights_list[-1][2] * gbm.predict(test[features]) +\
weights_list[-1][3] * rule_res

CPU times: user 2.22 s, sys: 36 ms, total: 2.25 s
Wall time: 846 ms


In [912]:
test[['loc_id', 'time_stamp', 'num_of_people']].to_csv('submit/submit_gbm_normal_lgb.csv', index=False)